In [1]:
import numpy as np
import quaternion

from intrepid_environment.quadcopter_environment import QuadcopterController

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
# initialize RPC Drone Controller
DT_MS = 10

quadcopter_rpc_controller = QuadcopterController(DT_MS)

Setup trajectory for drone to follow.

In [4]:
from quadcopter_control.trajectory import Trajectory, TrajectoryPoint

In [5]:
async def get_trajectory_with_initial_position():
    sim_time = await quadcopter_rpc_controller.simulation_time()
    initial_orientation = quadcopter_rpc_controller.rotation_angles()
    initial_attitude = quaternion.from_euler_angles(*initial_orientation)
    initial_omega = quadcopter_rpc_controller.angular_velocity()
    initial_position = quadcopter_rpc_controller.position()
    initial_linear_velocity = quadcopter_rpc_controller.linear_velocity()
    initial_acceleration = quadcopter_rpc_controller.acceleration()

    starting_point = TrajectoryPoint(
        time=sim_time,
        position=initial_position,
        velocity=initial_linear_velocity,
        accel=initial_acceleration,
        attitude=initial_attitude,
        omega=initial_omega,
    )

    trajectory = Trajectory()
    trajectory.add_point(starting_point)

    return trajectory

In [6]:
def get_next_trajectory_point(trajectory, sim_time, dt, dp):
    pt = trajectory.next_trajectory_point((sim_time + dt) / 1000.0)
    pt.position += dp
    return pt

In [7]:
trajectory_time_offset = 0
dp = np.zeros(3)

In [8]:
from quadcopter_control.controller import QuadcopterController, calculate_drone_moment_of_inertia

In [9]:
DRONE_HEIGHT = 0.028
DRONE_RADIUS = 0.05
DRONE_MASS = 0.041

In [10]:
(Ixx, Iyy, Izz) = calculate_drone_moment_of_inertia(DRONE_HEIGHT, DRONE_RADIUS, DRONE_MASS)
(Ixx, Iyy, Izz)

(2.830366666666667e-05, 2.830366666666667e-05, 5.125000000000001e-05)

## Scenario 0: Zero Gains

Test whether controller runs without controller gains set.

In [11]:
trajectory = await get_trajectory_with_initial_position()

In [12]:
quadcopter_controller = QuadcopterController(
    DT_MS / 1000.0,
    DRONE_MASS,
    Ixx, Iyy, Izz,
    max_tilt_angle=0.52,
    max_ascent_rate=4.0
)

In [13]:
trajectory = await get_trajectory_with_initial_position()

async def control_step():
    sim_time = await quadcopter_rpc_controller.simulation_time()
    motor_commands = quadcopter_controller.run_control(
        traj_pt=trajectory.next_trajectory_point((sim_time+DT_MS)/1000.0),
        est_pos=quadcopter_rpc_controller.position(),
        est_vel=quadcopter_rpc_controller.linear_velocity(),
        est_omega=quadcopter_rpc_controller.angular_velocity(),
        est_att=quaternion.from_euler_angles(
            *quadcopter_rpc_controller.rotation_angles()
        ),
    )
    # scale motor commands
    motor_commands *= 170
    quadcopter_rpc_controller.control_motors(*motor_commands)
    quadcopter_rpc_controller.step()

In [14]:
# await control_step()

## Scenario 1: Altitude Control

Tune gains until drone stays in place. on a desired height.

In [15]:
trajectory = await get_trajectory_with_initial_position()

In [16]:
# add 500 datapoints so drone stays in place
sim_time = await quadcopter_rpc_controller.simulation_time()

n_episodes = 1000

for i in range(0, n_episodes):
    trajectory.add_point(TrajectoryPoint(
        time=(sim_time + (i*DT_MS) / 1000.0),
        position=np.array([0.0, 0.0, 5.0]),
        velocity=np.zeros(3),
        accel=np.zeros(3),
        attitude=np.quaternion(1.0, 0.0, 0.0, 0.0),
        omega=np.array([0.0, 0.0, 0.0])
    ))

Set controller gains. Test only integrated error of altitude controller.

In [17]:
async def control_step(dt):
    sim_time = await quadcopter_rpc_controller.simulation_time()
    traj_pt = trajectory.next_trajectory_point(sim_time / 1000000.0 + dt)

    (t_pos, t_vel, t_acc) = traj_pt.position, traj_pt.velocity, traj_pt.accel

    est_pos = quadcopter_rpc_controller.position()
    est_vel = quadcopter_rpc_controller.linear_velocity()
    est_att = quaternion.from_euler_angles(
        *quadcopter_rpc_controller.rotation_angles()
    )

    thrust_cmd = quadcopter_controller.altitude_control(
        t_pos[2],
        t_vel[2],
        est_pos[2],
        est_vel[2],
        est_att,
        t_acc[2],
        dt
    )

    max_motor_thrust = 0.7
    min_motor_thrust = 0.3

    thrust_margin = 0.1 * (max_motor_thrust - min_motor_thrust)
    clipped_thrust = np.clip(thrust_cmd, min_motor_thrust, max_motor_thrust)

    motor_commands = quadcopter_controller.generate_motor_commands(clipped_thrust, np.zeros(3))
    motor_commands *= 120.0

    quadcopter_rpc_controller.control_motors(*motor_commands)
    quadcopter_rpc_controller.step()

    return thrust_cmd, clipped_thrust, motor_commands

In [18]:
async def tune_altitude_control(
        kp_pqr=np.array([95.0, 95.0, 6.0]), 
        ki_pos_z=8.0, 
        kp_pos_z=5.0,
        kp_vel_z=2.0,
        kappa=2.0,):
    quadcopter_rpc_controller.restart()
    quadcopter_controller.set_gains(kp_pqr, ki_pos_z, kp_pos_z,kp_vel_z, kappa)
    prev_sim_time = await quadcopter_rpc_controller.simulation_time()

    altitudes = []
    thrusts = []
    mean_motor_commands = []

    for _ in range(5000):
        est_pos = quadcopter_rpc_controller.position()
        altitudes.append(est_pos[2])
        sim_time = await quadcopter_rpc_controller.simulation_time()
        dt = (sim_time - prev_sim_time) / 1000000.0
        print("dt", dt)
        prev_sim_time = sim_time
        (thrust_cmd, clipped_thrust, motor_commands) = await control_step(dt)
        thrusts.append(clipped_thrust)
        mean_motor_commands.append(np.mean(motor_commands))

    return altitudes, thrusts, mean_motor_commands

In [19]:
import matplotlib.pyplot as plt

In [20]:
(altitudes, thrusts, mean_motor_commands) = await tune_altitude_control(ki_pos_z=5, kp_pos_z=0, kp_vel_z=0)

quadcopter_rpc_controller.control_motors(0, 0, 0, 0)

plt.plot(altitudes)
plt.show()

plt.plot(thrusts)
plt.show()

plt.plot(mean_motor_commands)
plt.show()

dt 0.03125
dt 0.078125
dt 0.0625
dt 0.078125
dt 0.078125
dt 0.078125
dt 0.078125
dt 0.078125
dt 0.078125
dt 0.078125
dt 0.078125
dt 0.078125
dt 0.078125
dt 0.078125
dt 0.09375
dt 0.078125
dt 0.078125
dt 0.078125
dt 0.0625
dt 0.078125
dt 0.09375
dt 0.078125
dt 0.078125
dt 0.078125
dt 0.078125


CancelledError: 